In [1]:
from FinMind.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import requests

dl = DataLoader()
df = dl.taiwan_stock_daily(stock_id='0050', start_date='2003-01-01', end_date='2023-02-25')

df = df.rename(columns={"date": "Date"})
df.set_index("Date" , inplace=True)
df = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
df = df.rename(columns={"Trading_Volume": "Volume"})
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))

In [2]:
token = pd.read_csv(r"C:\Users\Wade\Desktop\Jupyter_Python\finMindToken.txt")

In [3]:
api = DataLoader()
api.login_by_token(api_token=token)
# api.login(user_id='user_id',password='password')
df = api.taiwan_futures_tick(
    futures_id='MTX',
    date='2024-05-11'
)

In [4]:
df

,contract_date,date,futures_id,price,volume
0,202405,2024-05-11 00:00:00,MTX,20941.0,2
1,202405,2024-05-11 00:00:00,MTX,20941.0,4
2,202405,2024-05-11 00:00:00,MTX,20941.0,2
3,202405,2024-05-11 00:00:00,MTX,20941.0,2
4,202405,2024-05-11 00:00:00,MTX,20941.0,2
...,...,...,...,...,...
13105,202406,2024-05-11 04:59:54,MTX,20949.0,2
13106,202405,2024-05-11 04:59:57,MTX,20946.0,2
13107,202406,2024-05-11 04:59:57,MTX,20948.0,2
13108,202406,2024-05-11 04:59:57,MTX,20948.0,2
